In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath(".."))

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_blobs

from preprocessing.ohe import OneHotEncoder
from neighbors.knn_classifier import KNeighborsClassifier
# from linear_models.classification.logistic_regression import LogisticRegression
# from linear_models.regression.linear_regression import LinearRegression
# from linear_models.classification.least_squares_classifier import LeastSquareClassifier
# from classification.perceptron import Perceptron

In [2]:
OneHotEncoder([1,2,3,1])

IndexError: index 3 is out of bounds for axis 0 with size 3

In [ ]:
# X, y = make_blobs(n_samples=10, centers=3, n_features=2)
# x_pred, y = make_blobs(n_samples=1, centers=3, n_features=2)

In [ ]:
# knn = KNeighborsClassifier(2, 'manhattan')
# knn.fit(X, y)

In [ ]:
# knn.euclidian_distance(x_pred)

In [ ]:
# print(knn.distances)

In [ ]:
# from sklearn.datasets import make_regression
# X, y = make_regression(n_samples=100, n_features=2)
# lr = LinearRegression()
# lr.fit(X,y)

In [ ]:
# from sklearn.datasets import make_classification
# X, y = make_classification(n_samples=100, n_features=2, n_redundant=0, class_sep=2)
# y = y.reshape(-1,1)
# lr = LogisticRegression(threshold=0.5, optimizer = 'SGD')
# lr.fit(X,y,verbose=False, epochs=20, lr=0.01, batch_size = 5,  fit_intercept = True)
# y_hat = lr.predict(X)

In [ ]:
# sns.scatterplot(data = X, x = X[:,0], y = X[:,1], hue = y.flatten())

In [ ]:
# sns.scatterplot(data = X, x = X[:,0], y = X[:,1], hue = y_hat.flatten())

In [ ]:
# plt.scatter(X, y)
# y_hat = lin.predict(X)
# plt.scatter(X, y_hat)